Authors: Karen Stengel, Jakidxav
    
In this Jupyter Notebook, we will show how you can load in a pre-trained model and predict on your test set data.

In [0]:
#imports
from contextlib import redirect_stdout
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import seaborn as sns

import numpy as np
import pickle
import os
import datetime
import time
import random
import sys

from keras.models import Sequential, Model, save_model, load_model
import keras.backend as K
from keras.layers import Dense, Activation, Conv2D, Input, AveragePooling2D, MaxPooling2D, Flatten, LeakyReLU, TimeDistributed, LSTM, Dropout, BatchNormalization
from keras.metrics import binary_accuracy
from keras.losses import  binary_crossentropy
from keras.regularizers import l2
from keras.optimizers import SGD, Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model
import sklearn.metrics as skm
from tensorflow import metrics as tf

#will allow for files to have their text treated as text in illustrator
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

from plotting import *

Using TensorFlow backend.


In [0]:
#path for data output. each file should contain all used params after training and metrics
#outputDir = "/Users/joshuadr/Desktop/final/testset/"
outputDir = "/Users/jakidxav/Desktop/final/testset/"
X_val_filename40 = '/Users/jakidxav/Desktop/Testing/50_lead/X_val/X_val.txt'

Y_val_filename = '/Users/jakidxav/Desktop/Testing/Y/Y_val/station0/Y_val.txt'
Y_val_week_filename = '/Users/jakidxav/Desktop/Testing/Y_val_week.txt'

#load in data and labels
test_data40, test_label = load_data(X_val_filename40, Y_val_filename)
    
with open(Y_val_week_filename, 'rb') as k:
    week_test_label = pickle.load(k)

#reshape all data files to have the shape we need
test_data40_2 = test_data40.reshape(-1,120,340,1)

In [0]:
#load in model, cnn 40 lead
cnn40 = load_model('/Users/jakidxav/Desktop/180725__50_4.2_.h5')

#make predictions on validation (or test) set
cnn40_test_pred = cnn40.predict(test_data40_2).ravel()

#create roc plot
cnn40_fpr_test, cnn40_tpr_test, cnn40_thresholds_test = skm.roc_curve(week_test_label,cnn40_test_pred)
cnn40_auroc = skm.auc(cnn40_fpr_test,cnn40_tpr_test)

#get scoring data
cnn40_score = cnn40.evaluate(test_data40_2, week_test_label, verbose=1)

#make roc plot
makePlots(outputDir + "180725__50_4.2_week", "Week CNN 50", cnn40_fpr_test, cnn40_tpr_test, cnn40_test_pred)

#open text file, write scores and metricss to file
cnn40_file = open(outputDir + "180725__50_4.2_week.txt", "w+")
cnn40_file.write("%s: %.2f%%\n" % (cnn40.metrics_names[1], cnn40_score[1]*100))
cnn40_file.write("%s: %.2f%%" % ("AUROC score", cnn40_auroc))
cnn40_file.write("\n\n")
with redirect_stdout(cnn40_file):
    cnn40.summary()

240/240 [==============================] - 0s 535us/step


In [0]:
#let's take a look at what our predictions look like
print(cnn40_test_pred)

[  7.26511684e-10   5.02113305e-14   1.42716417e-05   0.00000000e+00
   0.00000000e+00   0.00000000e+00   9.76970674e-13   1.47214985e-06
   7.02539238e-16   3.07362601e-02   9.95817213e-22   2.81509655e-14
   9.72947486e-11   4.59839903e-13   2.53024722e-17   1.21790310e-18
   2.40296900e-01   0.00000000e+00   0.00000000e+00   1.75485549e-28
   3.27361120e-23   8.88140638e-30   0.00000000e+00   0.00000000e+00
   0.00000000e+00   1.44333085e-25   0.00000000e+00   1.01953667e-07
   1.37676857e-03   1.44677542e-05   1.84331715e-01   9.37137485e-01
   2.61113733e-01   7.24635601e-01   6.50877890e-04   7.58507199e-34
   4.21376944e-01   1.42291740e-01   6.37418747e-01   1.73448542e-26
   3.86854238e-09   5.40832365e-29   9.24756408e-01   9.30204570e-01
   8.11722100e-01   1.16974287e-01   7.05080986e-01   4.20663625e-01
   9.92746890e-01   9.48250949e-01   8.55045095e-02   5.06218340e-33
   4.87126262e-26   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.38278592e-23   2.20926553e-01